# IG09 - Personal a honorarios (proveedores) por años.

El indicador IG09 verifica la cantidad de documentos que emiten proveedores personas naturales a la organización durante los periodos de dos y un año. El área responsable es la Gerencia de Abastecimiento.

El siguiente código es para ejecutar el indicador de Auditoría Continua IG09 - Personal a honorarios (proveedores) por años.

 - ***Fecha creación: 15.12.2020***

## Importar librerías
---

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
import string

## Utilidades
---

In [ ]:
def clean_blankspace(df):
    #eliminar espacios en nombres de columnas
    columns = list(df.columns)
    columns = [w.replace(' ','') for w in columns]
    df.columns = columns
    #Eliminar espacios en cada celda del dataframe
    for columna in list(df.columns):
        if df.dtypes[columna] == np.dtype('O'):
            df[columna] = df[columna].str.replace(' ','')
    return df

def clean_columns(txt):
    df = txt.drop(columns=txt.columns[0:2]).drop(columns=txt.columns[-1]).dropna(how='all').reset_index(drop=True)
    return df

def num_format(df,col):
    df[col] = df[col].str.replace('.','').str.replace(',','.').astype(float)
    if (col=='DMBTR') or (col=='ERFMG') or (col=='VERPR'):
        df[col] = df[col]*100
    return df

def conversion_clp(waers,rlwrt):
    if waers == 'CLP':
        valor = rlwrt*100
    if waers == 'EUR':
        valor = rlwrt*906
    if waers == 'GBP':
        valor = rlwrt*1004.62
    if waers == 'UF':
        valor = (rlwrt/1000)*28915
    if waers == 'USD':
        valor = rlwrt*766
    if waers == 'UTM':                 
        valor = (rlwrt/100)*50674
    if waers == 'AUD':   
        valor = rlwrt*500.11
    if waers == 'CAD':
        valor = rlwrt*600.8
    if (waers == '') or (waers == 'nan'):
        valor = rlwrt
    return valor

def str_format(df,col):
    
    #Elimina letras, sinbolos de puntuacion, u otros simbolos de columnas numericas
    
    punctuation = list(string.punctuation)   # lista de simbolos de puntuacion      
    letters = list(string.ascii_letters)     # Lista de letras mayusculas y minusculas        
    symbols = ['^','°']                      # Lista de simbolos. Si aparece uno nuevo, habría que agregarlo a la lista!

    for i in range(len(punctuation)):
        df[col] = df[col].str.replace(punctuation[i],'')   # Elimina letras

    for i in range(len(letters)):
        df[col] = df[col].str.replace(letters[i],'')       # Elimina simbolos de puntuación
    
    for i in range(len(symbols)):
        df[col] = df[col].str.replace(symbols[i],'')       # Elimina otros simbolos

    df = df[df[col] != '']                                 # Elimina celdas sin contenido
    
    return df

## Cargar tablas
---

In [ ]:
#input_path = 'C:/projects/auditoria_continua/data/input/current/'
#output_path = 'C:/projects/auditoria_continua/data/output/current/'

In [ ]:
rbkp_df = pd.read_csv('rbkp.txt', sep='|', header=3, encoding='latin1', low_memory=False)
lfa1_df = pd.read_csv('lfa1.txt', sep='|', header=3, encoding='latin1', low_memory=False)

## Limpieza
---

**Eliminar columnas y filas nulas**

In [ ]:
rbkp_df = clean_columns(rbkp_df)
lfa1_df = clean_columns(lfa1_df)

**Eliminar espacios en columnas y cada celda**

In [ ]:
rbkp_df = clean_blankspace(rbkp_df)
lfa1_df = clean_blankspace(lfa1_df)

**Cambio de formato de columnas**

In [ ]:
rbkp_df = str_format(rbkp_df,'XBLNR')                              # elimina letras o simbolos de columnas numericas  
rbkp_df = num_format(rbkp_df,'RMWWR')                              # reemplaza "," por "", y "," por "."
rbkp_df = rbkp_df.replace(r'^\s*$', np.nan, regex=True)            # reemplaza valores vacios con np.nan

**Establecer esquema de datos**

In [ ]:
schema_rbkp = {'BELNR': 'float64',     # Doc_material 
                'GJAHR': str,          # Año 
                'BLART': str,          # Clase_doc_material 
                'BUDAT': str,          # Fecha_contabilidad 
                'USNAM': str,          # Usuario 
                'XBLNR': 'float64',    # Referencia 
                'LIFNR': 'float64',    # Proveedor 
                'WAERS': str,          # Moneda 
                'RMWWR': 'float64'}    # Monto_bruto

schema_lfa1 = {'LIFNR': 'float64',     # Proveedor
                'NAME1': str,          # Nombre_proveedor
                'SORTL': str,          # Rut_proveedor
                'BRSCH': str,          # Tipo_proveedor
                'KTOKK': str}          # Grupo_de_cuentas

rbkp_df = rbkp_df.astype(schema_rbkp)
lfa1_df = lfa1_df.astype(schema_lfa1)

**Cambio de formato de columnas en su respectiva moneda**

In [ ]:
rbkp_df['RMWWR_CLP'] = rbkp_df.apply(lambda x: conversion_clp(x['WAERS'],x['RMWWR']),axis=1)

**Eliminar columnas que no se utilizan en este indicador**

In [ ]:
rbkp_df.drop(columns= ['RMWWR'], inplace= True)

## Ejecutar indicador
---

**Cruzar ambas tablas**

In [ ]:
df = pd.merge(rbkp_df, lfa1_df, on='LIFNR', how='inner')
df = df.drop_duplicates()

**Filtrar los tipos de documentos a analizar**

In [ ]:
df = df[(df['BLART']=='BE') |
        (df['BLART']=='BM') |
        (df['BLART']=='KA') |
        (df['BLART']=='KT') |
        (df['BLART']=='KU') |
        (df['BLART']=='KV') |
        (df['BLART']=='KK') |
        (df['BLART']=='KR') |
        (df['BLART']=='KS') |
        (df['BLART']=='KW') |
        (df['BLART']=='KX')]

**Hacer filtro por rut de personas naturales o que sean EIRL**

In [ ]:
p_natural = df[(df['SORTL'].str.startswith('1')) | 
                 (df['SORTL'].str.startswith('2')) |
                 (df['SORTL'].str.startswith('3'))].reset_index(drop=True)

In [ ]:
eirl = df[(df['NAME1'].str.contains('EIRL')) |
          (df['NAME1'].str.contains('E.I.R.L'))].reset_index(drop=True)

In [ ]:
pnat_eirl = pd.concat([p_natural, eirl], axis='index')

**Columnas Auxiliares**

In [ ]:
mes = []
for i in pnat_eirl['BUDAT']:
    aux = i[3:5]
    mes.append(aux)

pnat_eirl['Mes'] = mes

In [ ]:
dias = []

for i in pnat_eirl['BUDAT']:
    dia = i[0:2]
    dias.append(dia)
    
pnat_eirl['Dia'] = dias

In [ ]:
pnat_eirl['fecha'] = pnat_eirl['GJAHR'].str[:4] + pnat_eirl['Mes'] + pnat_eirl['Dia']
pnat_eirl['fecha'] = pnat_eirl['fecha'].astype(int)

In [ ]:
pnat_eirl.head()

In [ ]:
pnat_eirl['fecha'].max()

**Calcular cantidad de documentos dentro de 1 año y de 2 años**

In [ ]:
fecha_anio = int((datetime.now() - timedelta(days=366)).strftime('%Y%m%d'))
fecha_anio2 = int((datetime.now() - timedelta(days=734)).strftime('%Y%m%d'))
fecha_anio

In [ ]:
print(type(fecha_anio), fecha_anio)

In [ ]:
print(type(fecha_anio2), fecha_anio2)

2 Años (24 meses)

In [ ]:
meses24_df = pnat_eirl[pnat_eirl['fecha']>=fecha_anio2].reset_index(drop=True)

In [ ]:
print(meses24_df['fecha'].max(),meses24_df['fecha'].min())

In [ ]:
meses24_df['ano_mes'] = meses24_df['fecha'].astype(str).str[0:6]

In [ ]:
meses24_group = meses24_df.groupby(['LIFNR', 'SORTL', 'NAME1', 'ano_mes']).agg({'BELNR':'count'}).reset_index().sort_values(by='LIFNR')

In [ ]:
mask = meses24_group.LIFNR.duplicated(keep=False)
meses24_group['duplicado'] = meses24_group.LIFNR.mask(mask, 0)
meses24_group = meses24_group.drop(meses24_group[meses24_group['duplicado']!=0].index).sort_values(by='LIFNR')
meses24_group = meses24_group.drop(['duplicado'], axis=1)

In [ ]:
meses24_group

**Indicador**

In [ ]:
 meses24_group.groupby(['LIFNR', 'SORTL', 'NAME1']).agg({'ano_mes':'count', 'BELNR':'sum'}).sort_values(by='LIFNR').reset_index()

In [ ]:
 meses24_group.groupby(['LIFNR', 'SORTL', 'NAME1']).agg({'ano_mes':'count', 'BELNR':'sum'}).sort_values(by='LIFNR').reset_index()

In [ ]:
indicador_1 = meses24_group.groupby(['LIFNR', 'SORTL', 'NAME1']).agg({'ano_mes':'count', 'BELNR':'sum'}).sort_values(by='LIFNR').reset_index()
indicador_1 = indicador_1[indicador_1['ano_mes']>=12].reset_index(drop=True)
indicador_1.columns = ['Numero_Proveedor',
                        'RUT_proveedor',
                        'Nombre_Proveedor',
                        'N_meses_con documento',
                        'Cantidad_documentos']

In [ ]:
indicador_1

In [ ]:
detalle_1 = pd.merge(pnat_eirl, indicador_1, left_on='LIFNR', right_on='Numero_Proveedor', how='inner')
detalle_1 = detalle_1[['LIFNR', 'SORTL', 'NAME1', 'BELNR', 'BLART', 'XBLNR', 'BUDAT', 'RMWWR_CLP']].sort_values(by='LIFNR').reset_index(drop=True)
detalle_1.columns = ['Numero_Proveedor',
                     'RUT_Proveedor',
                     'Nombre_Proveedor',
                     'Documento_material',
                     'Clase_documento',
                     'Referencia',
                     'fecha_ingreso',
                     'Monto_bruto_clp']

In [ ]:
detalle_1

## Guardar en Excel

In [ ]:
nombre_archivo = 'ac12_' + datetime.now().strftime("%d-%m-%y_%Hh%Mm") + '.xlsx'
writer = pd.ExcelWriter(nombre_archivo, engine='xlsxwriter')

indicador_1.to_excel(writer, sheet_name='Proveedores_12_meses_o_mas')
detalle_1.to_excel(writer, sheet_name='Detalle_12_meses_o_mas')

writer.save()

In [ ]:
meses24_group[meses24_group['NAME1']=='RAFAELENRIQUEGAETEBARRERA'].sort_values(by='ano_mes')